**SCRAPING  DATASET USING BEAUTIFULSOUP**

In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def auto_Scrapper_Class(html_tag, course_case, tag_class):
    for i in range(1, 100):
        url = "https://www.coursera.org/courses?page=" + str(i) + "&index=prod_all_products_term_optimization"
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        courses = soup.find_all(html_tag, class_=tag_class)
        for course in courses:
            # Course title
            course_title = course.find('h3', class_='cds-CommonCard-title').get_text().strip()
            course_case['course_title'].append(course_title)
            
            # Organization offering the course
            course_organization = course.find('p', class_='cds-ProductCard-partnerNames').get_text().strip()
            course_case['course_organization'].append(course_organization)
            #SKILLS
          #SKILLS
            course_skills_element = course.find('div', class_='cds-CommonCard-bodyContent')
            course_skills = course_skills_element.get_text(strip=True) if course_skills_element else 'N/A'
            course_skills=course_skills.split(':')
            skills=course_skills[1] if len(course_skills) >1 else 'N/A'
            course_case['course_skills'].append(skills)
            # Extracting metadata
            metadata = course.find('div', class_='cds-CommonCard-metadata').get_text(strip=True)
            metadata_parts = metadata.split(' · ')
            
            # Certificate type
            certificate_type = metadata_parts[1] if len(metadata_parts) > 1 else 'N/A'
            course_case['course_Certificate_type'].append(certificate_type)
            
            # Difficulty
            difficulty = metadata_parts[0] if len(metadata_parts) > 0 else 'N/A'
            course_case['course_difficulty'].append(difficulty)
            
            # Rating
            product_reviews_tag = course.find('div', class_='product-reviews')
            course_rating_tag = product_reviews_tag.find('p', class_='css-2xargn') if product_reviews_tag else None
            course_rating = course_rating_tag.get_text().strip() if course_rating_tag else 'None'
            course_case['course_rating'].append(course_rating)
            
            # Students enrolled
            
            course_students_tag = product_reviews_tag.find('p', class_='css-vac8rf') if product_reviews_tag else None
            course_students_enrolled = course_students_tag.get_text().strip() if course_students_tag else 'None'
            if course_students_enrolled != 'None':
                course_students_enrolled = course_students_enrolled.replace('(', '') 
                course_students_enrolled = course_students_enrolled.split()[0] 
            course_case['course_students_enrolled'].append(course_students_enrolled)


# Initialize dictionary to store course data
course_data = {
    'course_title': [],
    'course_organization': [],
    'course_skills':[],
    'course_Certificate_type': [],
    'course_rating': [],
    'course_difficulty': [],
    'course_students_enrolled': []
}

auto_Scrapper_Class('div', course_data, 'cds-ProductCard-content')

courses_df = pd.DataFrame(course_data)
courses_df = courses_df.replace(['None', 'N/A'], pd.NA)
courses_df = courses_df.dropna()

# Save to CSV
courses_df.to_csv('Coursera_courses.csv', index=False)